##### Copyright 2019 The TensorFlow Authors.

In [11]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow 2 quickstart for beginners

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Load a prebuilt dataset.
1. Build a neural network machine learning model that classifies images.
2. Train this neural network.
3. Evaluate the accuracy of the model.

This tutorial is a [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) notebook. Python programs are run directly in the browser—a great way to learn and use TensorFlow. To follow this tutorial, run the notebook in Google Colab by clicking the button at the top of this page.

1. In Colab, connect to a Python runtime: At the top-right of the menu bar, select *CONNECT*.
2. Run all the notebook code cells: Select *Runtime* > *Run all*.

## Set up TensorFlow

Import TensorFlow into your program to get started:

In [12]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.10.0


If you are following along in your own development environment, rather than [Colab](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb), see the [install guide](https://www.tensorflow.org/install) for setting up TensorFlow for development.

Note: Make sure you have upgraded to the latest `pip` to install the TensorFlow 2 package if you are using your own development environment. See the [install guide](https://www.tensorflow.org/install) for details.

## Load a dataset

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the sample data from integers to floating-point numbers:

In [13]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## Build a machine learning model

Build a `tf.keras.Sequential` model by stacking layers.

In [14]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example, the model returns a vector of [logits](https://developers.google.com/machine-learning/glossary#logits) or [log-odds](https://developers.google.com/machine-learning/glossary#log-odds) scores, one for each class.

In [15]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.57927966,  0.12368314,  0.11630702,  0.26439863,  1.2024875 ,
         0.01823629,  0.38258526, -0.5829386 , -0.28073108, -0.40766913]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to *probabilities* for each class: 

In [16]:
tf.nn.softmax(predictions).numpy()

array([[0.04704666, 0.09502144, 0.09432314, 0.10937891, 0.27947336,
        0.08551192, 0.12310096, 0.04687483, 0.06341424, 0.05585452]],
      dtype=float32)

Note: It is possible to bake the `tf.nn.softmax` function into the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output. 

Define a loss function for training using `losses.SparseCategoricalCrossentropy`, which takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [17]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [18]:
loss_fn(y_train[:1], predictions).numpy()

2.4590995

Before you start training, configure and compile the model using Keras `Model.compile`. Set the [`optimizer`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) class to `adam`, set the `loss` to the `loss_fn` function you defined earlier, and specify a metric to be evaluated for the model by setting the `metrics` parameter to `accuracy`.

In [19]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## Train and evaluate your model

Use the `Model.fit` method to adjust your model parameters and minimize the loss: 

In [20]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
  70/1875 [>.............................] - ETA: 1s - loss: 1.1712 - accuracy: 0.6696  

2022-11-29 14:36:59.353357: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1875/1875 [==============================] - 1s 716us/step - loss: 0.3039 - accuracy: 0.9119
Epoch 2/5
1875/1875 [==============================] - 1s 731us/step - loss: 0.1459 - accuracy: 0.9567
Epoch 3/5
1875/1875 [==============================] - 1s 786us/step - loss: 0.1081 - accuracy: 0.9672
Epoch 4/5
1875/1875 [==============================] - 1s 733us/step - loss: 0.0881 - accuracy: 0.9724
Epoch 5/5
1875/1875 [==============================] - 1s 729us/step - loss: 0.0766 - accuracy: 0.9757


The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [21]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0781 - accuracy: 0.9769 - 192ms/epoch - 615us/step


[0.07806853950023651, 0.9768999814987183]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [22]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [23]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[1.2988227e-07, 5.3429027e-08, 4.2169479e-05, 4.6582305e-04,
        1.0933346e-09, 1.3477770e-06, 5.0676715e-13, 9.9940658e-01,
        4.1892322e-06, 7.9739613e-05],
       [1.6217358e-07, 5.6462835e-05, 9.9992096e-01, 1.3247374e-05,
        2.9637672e-15, 1.7632406e-07, 2.2889363e-07, 3.4029364e-12,
        8.6618456e-06, 3.6783812e-14],
       [2.1977684e-08, 9.9938369e-01, 7.7537952e-05, 1.9488065e-05,
        4.6634290e-05, 9.6164513e-07, 5.4353241e-06, 2.9659283e-04,
        1.6862742e-04, 9.2643097e-07],
       [9.9963856e-01, 2.1366239e-07, 5.0670082e-05, 1.0514485e-06,
        2.0110554e-06, 5.5215492e-05, 1.8440627e-05, 1.2965393e-04,
        3.2577746e-06, 1.0093386e-04],
       [2.3008379e-05, 1.6165014e-07, 6.4268470e-06, 2.1790147e-07,
        9.9934703e-01, 4.4815494e-07, 4.9992155e-05, 4.9164839e-05,
        2.3683726e-05, 4.9989467e-04]], dtype=float32)>

## Conclusion

Congratulations! You have trained a machine learning model using a prebuilt dataset using the [Keras](https://www.tensorflow.org/guide/keras/overview) API.

For more examples of using Keras, check out the [tutorials](https://www.tensorflow.org/tutorials/keras/). To learn more about building models with Keras, read the [guides](https://www.tensorflow.org/guide/keras). If you want learn more about loading and preparing data, see the tutorials on [image data loading](https://www.tensorflow.org/tutorials/load_data/images) or [CSV data loading](https://www.tensorflow.org/tutorials/load_data/csv).
